Changelog
- 30 Sep 2019
    - Creation
    - Goal is to do MOBIL experimentation to demonstrate lane changes
    - Idea flow
        - Look at some videos to select a situation with enough spacing to make lane change
        - Select a vehicle id that is on a slow lane
        - Make it want to make a lange change by some selection of parameters

In [1]:
# using packages
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Reel

In [11]:
# define global things: timestep, roadway, traj
timestep_ngsim = 0.1;

In [37]:
# overlay: IDOverlay
"""
    IDOverlay
Display the ID on top of each entity in a scene.
# Fields
- `color::Colorant`
- `font_size::Int64`
"""
mutable struct IDOverlay <: SceneOverlay
    color::Colorant
    font_size::Int
end

function AutoViz.render!(rendermodel::RenderModel, overlay::IDOverlay, scene::Scene, 
                            env::E) where E
    font_size = overlay.font_size
    for veh in scene
        add_instruction!(rendermodel, render_text, ("$(veh.id)", veh.state.posG.x, 
                        veh.state.posG.y, font_size, overlay.color), incameraframe=true)
    end
    return rendermodel
end

In [2]:
# function: generate roadway and place cars
"""
    function init_place_cars(lane_place_array;road_length = 1000.0)
- Place cars on a straight roadway of `road_length` according to elems in `lane_place_array`

# Examples
```julia
pos_vel_array_1 = [(200.,30.),(215.,0.),(220.,0.)]
pos_vel_array_2 = [(200.,0.),(215.,0.),(220.,20.)]
pos_vel_array_3 = [(215.,0.),(225.,10.),(230.,0.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2,pos_vel_array_3]
scene,roadway = init_place_cars(lane_place_array)
```
"""
function init_place_cars(lane_place_array;road_length = 1000.0)
    num_lanes = length(lane_place_array)
    roadway = gen_straight_roadway(num_lanes,road_length)
    scene = Scene()

    id = 1
    for i in 1:num_lanes
        for j in 1:length(lane_place_array[i])
            veh_state = VehicleState(Frenet(roadway[LaneTag(1,i)],
                    lane_place_array[i][j][1]),roadway,
                lane_place_array[i][j][2])
            veh = Vehicle(veh_state,VehicleDef(),id)
            push!(scene,veh)
            id+=1
        end
    end
    return scene,roadway
end

init_place_cars

In [41]:
# function: make video from scene list
"""
    function scenelist2video
- Make video from a list of scenes

# Examples
```julia
scenelist2video(scene_list,roadway=roadway)
```
"""
function scenelist2video(scene_list;roadway,
    filename = "media/mobil/scene_to_video.mp4")
    frames = Frames(MIME("image/png"),fps = 10)
    
    # Loop over list of scenes and convert to video
    for i in 1:length(scene_list)
        scene_visual = render(scene_list[i],roadway,
        [IDOverlay(colorant"white",12)],
#         cam=FitToContentCamera(0.),
        cam = CarFollowCamera(1)
        )
        push!(frames,scene_visual)
    end
    print("Making video filename: $(filename)\n")
    write(filename,frames)
    return nothing
end

scenelist2video

In [32]:
# function: hallucinate scene list
"""
    function get_hallucination_scenes
- Hallucinate starting from `start_step` for `nsteps` using `models` and return a list of scenes
- Used by `plot_carwise_pos_vel` to assess position and velocity traces against ground truth

# Returns
- `halluc_scenes_list`: List containing the scenes starting with the ground truth scene at `start_step`

# Examples
```julia
scene_list = get_hallucination_scenes(start_scene,nsteps=100,models=models,
roadway=roadway);
```
"""
function get_hallucination_scenes(start_scene;nsteps,models,start_step=1,
        id_list=[],roadway,timestep=0.1,verbosity = false)
        # Setting up
    scene_halluc = start_scene
    halluc_scenes_list = []
#     scene_halluc = get_scene(start_step,traj) # Frame to start hallucination from
#     push!(halluc_scenes_list,deepcopy(scene_halluc))
    
    for (i,t) in enumerate(start_step:start_step+nsteps-1)
        
#         if !isempty(id_list) keep_vehicle_subset!(scene_halluc,id_list) end
        
        actions = Array{Any}(undef,length(scene_halluc))

            # Propagation of scene forward
        get_actions!(actions,scene_halluc,roadway,models)

        tick!(scene_halluc,roadway,actions,timestep)
        
        push!(halluc_scenes_list,deepcopy(scene_halluc))
    end 
    return halluc_scenes_list
end

get_hallucination_scenes

In [43]:
# script: Place cars on a straight roadway and generate driving video
pos_vel_array_1 = [(200.,29.),(215.,29.),(220.,29.)]
pos_vel_array_2 = [(200.,29.),(215.,29.),(220.,29.)]
pos_vel_array_3 = [(215.,29.),(225.,29.),(230.,29.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2,pos_vel_array_3]
scene,roadway = init_place_cars(lane_place_array)

models = Dict{Int64,DriverModel}()
for veh in scene
#     print("veh.id = $(veh.id)\n")
#     models[veh.id] = LatLonSeparableDriver(ProportionalLaneTracker(),IntelligentDriverModel())
    models[veh.id] = Tim2DDriver(timestep_ngsim,mlane = MOBIL(timestep_ngsim))
end


scene_list = get_hallucination_scenes(scene,nsteps=100,models=models,roadway=roadway)
scenelist2video(scene_list,roadway=roadway)

Making video filename: media/mobil/scene_to_video.mp4
